In [12]:
import findspark
findspark.init('/home/ubuntu/spark-2.2.0-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LogReg').getOrCreate()
data_folder = '/home/ubuntu/data/raw'

import os
os.getcwd()

'/home/ubuntu/notebooks'

In [9]:
from pyspark.ml.classification import LogisticRegression

In [14]:
file = '/sample_libsvm_data.txt'
data = spark.read.format('libsvm').load(data_folder + file)

In [16]:
data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



# Create LogReg

In [17]:
log_model =  LogisticRegression()

In [18]:
fitted_log_model =log_model.fit(data)

In [22]:
log_summary = fitted_log_model.summary
log_summary.predictions.printSchema()
log_summary.predictions.show(3)

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[39.9727764450750...|[1.0,4.3655982185...|       0.0|
|  1.0|(692,[158,159,160...|[-35.662380562160...|[3.25105944320044...|       1.0|
|  1.0|(692,[124,125,126...|[-39.336799621156...|[8.24603148700906...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 3 rows



In [25]:
lr_train , lr_test = data.randomSplit([.7,.3])

In [38]:
final_model = LogisticRegression()

fit_final = final_model.fit(lr_train)
prediction_labels = fit_final.evaluate(lr_test)

In [39]:
prediction_labels.predictions.show(2)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[61.1241269973138...|[1.0,2.8453057001...|       0.0|
|  0.0|(692,[121,122,123...|[98.5291259468250...|[1.0,1.6193633074...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 2 rows



# Evaluators

In [35]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

In [43]:
my_eval = BinaryClassificationEvaluator()
my_eval_roc_results = my_eval.evaluate(prediction_labels.predictions)

In [47]:
my_eval_roc_results

1.0